# Weekly Project 9

Today you are going to implement the last parts of the algorithm you started on monday. For reference you can see it below.

![title](algorithm_3.png)

It is a good idea to follow and track the steps in the algorithm in the below implementation. Only take one step at a time.

Once you have the algorithm up and running you can try with a larger dataset to see if your algorithm is able to maintain good accurracy over a longer distance. The larger dataset can be found here:
[Left images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EQu8kmGBDDROtGJ7IkZB2tQBJrxmgY9t8LVM_JuEi83TYw)
[Right images](https://dtudk-my.sharepoint.com/:u:/g/personal/evanb_dtu_dk/EcKI_zrXTvpMulizidCZm4oBLJcQ_LTV9Zs6oQFF74JTRQ)

In [1]:
import numpy as np
import cv2 as cv2
import glob

In [2]:
path = "C:/Users/shaia/Documents/Opgaveregning/AutoSys/4. semester/Perception for autonome systemer/Projekt/Projekt 9/"

def getK():
    return np.array([[7.188560e+02, 0.000000e+00, 6.071928e+02],
                     [0, 7.188560e+02, 1.852157e+02],
                     [0, 0, 1]])

def getTruePose():
    file = path + '00.txt'
    return np.genfromtxt(file, delimiter=' ', dtype=None)

<details>
  <summary>tips:</summary>

- [Feature Matching](https://docs.opencv.org/4.x/dc/dc3/tutorial_py_matcher.html)
- To get the keypoint of a certain match do: ```kp1[match.queryIdx].pt``` and ```kp2[match.trainIdx].pt```

In [3]:
def extract_keypoints_sift(img1, img2, K, baseline):
    """
    Use SIFT to detect keypoints and compute descriptors in both images,
    and find matches between them using knnMatch with k=2
    """
    sift = cv2.SIFT_create()

    """
    Get the best keypoints by applying the Lowes ratio test
    """
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Guard against no descriptors
    if des1 is None or des2 is None or len(des1) == 0 or len(des2) == 0:
        return np.zeros((0, 3)), np.zeros((0, 2), dtype=np.float32)

    bf = cv2.BFMatcher()
    matches_knn = bf.knnMatch(des1, des2, k=2)

    good = []
    for m_n in matches_knn:
        if len(m_n) < 2:
            continue
        m, n = m_n
        if m.distance < 0.75 * n.distance:
            good.append(m)

    if len(good) == 0:
        return np.zeros((0, 3)), np.zeros((0, 2), dtype=np.float32)

    # Extract matched point coordinates (left and right). Use queryIdx/trainIdx correctly.
    p1 = np.float32([kp1[m.queryIdx].pt for m in good])
    p2 = np.float32([kp2[m.trainIdx].pt for m in good])

    # Triangulation expects shape (2, N) for point arrays. Build projection matrices
    M_left = K.dot(np.hstack((np.eye(3), np.zeros((3, 1)))))
    M_rght = K.dot(np.hstack((np.eye(3), np.array([[-baseline, 0, 0]]).T)))

    p1_flip = np.vstack((p1.T, np.ones((1, p1.shape[0]))))
    p2_flip = np.vstack((p2.T, np.ones((1, p2.shape[0]))))

    P = cv2.triangulatePoints(M_left, M_rght, p1_flip[:2], p2_flip[:2])

    # Normalize homogeneous coordinates (P->Nx4  [N,4] is the normalizer/scale)
    P = P / P[3]
    land_points = P[:3]

    return land_points.T, p1

In [4]:
def featureTracking(prev_img, next_img, prev_points, world_points):
    """
    Use OpenCV to find the prev_points from the prev_img in the next_img
    Remember to remove points that could not be found from prev_points, next_points, and world_points
    hint: status == 1
    """
    params = dict(winSize=(21, 21),
                  maxLevel=3,
                  criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_img, next_img, prev_points, None, **params)

    if status is None:
        return None, None, None

    return world_points, prev_points, next_points

In [5]:
def playImageSequence(l_img_paths, r_img_paths, K):
    baseline = 0.54
    left_img = cv2.imread(l_img_paths[0], 0)
    right_img = cv2.imread(r_img_paths[0], 0)

    ##### ################################# #######
    ##### Get 3D points Using Triangulation #######
    ##### #########################################
    """
    Implement step 1.2 and 1.3
    Store the features in 'reference_2D' and the 3D points (landmarks) in 'landmark_3D'
    hint: use 'extract_keypoints_sift' above
    """
    landmark_3D, reference_2D = extract_keypoints_sift(left_img, right_img, K, baseline)

    # reference
    reference_img = left_img

    # Groundtruth for plot
    truePose = getTruePose()
    traj = np.zeros((600, 600, 3), dtype=np.uint8)
    maxError = 0

    for i in range(1, 101):
        print('image: ', i)
        curImage = cv2.imread(l_img_paths[i], 0)
        curImage_R = cv2.imread(r_img_paths[i], 0)

        ##### ############################################################# #######
        ##### Calculate 2D and 3D feature correspndances in t=T-1, and t=T  #######
        ##### #####################################################################
        """
        Implement step 2.2)
        Remember this is a part of a loop, so the initial features are already
        provided in step 1)-1.3) outside the loop in 'reference_2D' and 'landmark_3D'
        """
        landmark_3D, reference_2D, tracked_2Dpoints = featureTracking(reference_img, curImage, reference_2D, landmark_3D)


        ##### ####################################### #######
        ##### Calculate relative pose using PNPRansac #######
        ##### ###############################################
        """
        Implement step 2.3)
        """
        _, rvec, tvec, inliers = cv2.solvePnPRansac(landmark_3D, tracked_2Dpoints, K, None)


        ##### ####################################################### #######
        ##### Get Pose and Tranformation Matrix in world coordionates #######
        ##### ###############################################################
        rot, _ = cv2.Rodrigues(rvec)
        tvec = -rot.T.dot(tvec)  # coordinate transformation, from camera to world. What is the XYZ of the camera wrt World
        inv_transform = np.hstack((rot.T, tvec))  # inverse transform. A transform projecting points from the camera frame to the world frame

        ##### ################################# #######
        ##### Get 3D points Using Triangulation #######
        ##### #########################################
        # re-obtain the 3D points. 
        # hint: use 'extract_keypoints_sift' again
        """
        Implement step 2.4)
        """
        landmark_3D_new, reference_2D_new = extract_keypoints_sift(curImage, curImage_R, K, baseline)
        
        # Project the points from camera to world coordinates
        reference_2D = reference_2D_new.astype('float32')
        landmark_3D = inv_transform.dot(np.vstack((landmark_3D_new.T, np.ones((1, landmark_3D_new.shape[0])))))
        landmark_3D = landmark_3D.T

        ##### ####################### #######
        ##### Done, Next image please #######
        ##### ###############################
        reference_img = curImage

        ##### ################################## #######
        ##### START OF Print and visualize stuff #######
        ##### ##########################################
        # draw images
        draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100)
        true_x, true_y = int(truePose[i][3]) + 300, 600-(int(truePose[i][11]) + 100)

        curError = np.sqrt(
            (tvec[0] - truePose[i][3]) ** 2 +
            (tvec[1] - truePose[i][7]) ** 2 +
            (tvec[2] - truePose[i][11]) ** 2)
        
        if (curError > maxError):
            maxError = curError

        print(tvec[0],tvec[1],tvec[2], rvec[0], rvec[1], rvec[2])
        print([truePose[i][3], truePose[i][7], truePose[i][11]])
        
        text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
                                                                           float(tvec[2]))
        cv2.circle(traj, (draw_x, draw_y), 1, (0, 0, 255), 2)
        cv2.circle(traj, (true_x, true_y), 1, (255, 0, 0), 2)
        cv2.rectangle(traj, (10, 30), (550, 50), (0, 0, 0), cv2.FILLED)
        cv2.putText(traj, text, (10, 50), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1, 8)

        h1, w1 = traj.shape[:2]
        h2, w2 = curImage.shape[:2]
        vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
        vis[:h1, :w1, :3] = traj
        vis[:h2, w1:w1 + w2, :3] = np.dstack((np.dstack((curImage,curImage)),curImage))

        cv2.imshow("Trajectory", vis)
        k = cv2.waitKey(1) & 0xFF
        if k == 27: break


    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print('Maximum Error: ', maxError)
    ##### ################################ #######
    ##### END OF Print and visualize stuff #######
    ##### ########################################

In [6]:
# Load image paths
left_img_paths = sorted(glob.glob(path + 'left/*.png'))
right_img_paths = sorted(glob.glob(path + 'right/*.png'))

K = getK()

playImageSequence(left_img_paths, right_img_paths, K)

image:  1


C:\Users\shaia\AppData\Local\Temp\ipykernel_12264\799210300.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  draw_x, draw_y = int(tvec[0]) + 300, 600-(int(tvec[2]) + 100)
C:\Users\shaia\AppData\Local\Temp\ipykernel_12264\799210300.py:94: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  text = "Coordinates: x ={0:02f}m y = {1:02f}m z = {2:02f}m".format(float(tvec[0]), float(tvec[1]),
C:\Users\shaia\AppData\Local\Temp\ipykernel_12264\799210300.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operati

[1.82329052e-05] [-0.00512399] [0.67967976] [-0.00232375] [0.00345291] [-0.00248148]
[np.float64(-0.04690294), np.float64(-0.02839928), np.float64(0.8586941)]
image:  2
[-0.01307586] [-0.01126586] [1.37578278] [-0.00402097] [0.00743405] [-0.00070495]
[np.float64(-0.09374345), np.float64(-0.05676064), np.float64(1.716275)]
image:  3
[-0.03016759] [-0.01856186] [2.08729077] [-0.00553943] [0.01126298] [-0.00051381]
[np.float64(-0.1406429), np.float64(-0.08515762), np.float64(2.574964)]
image:  4
[-0.04941907] [-0.02864385] [2.81902306] [-0.00617717] [0.01613778] [0.00107271]
[np.float64(-0.1874858), np.float64(-0.1135202), np.float64(3.432648)]
image:  5
[-0.0643965] [-0.04071897] [3.56181263] [-0.00636199] [0.02075854] [3.72137855e-05]
[np.float64(-0.2343818), np.float64(-0.141915), np.float64(4.291335)]
image:  6
[-0.09233607] [-0.05029012] [4.32381516] [-0.00782698] [0.0252888] [0.00212084]
[np.float64(-0.2812195), np.float64(-0.1702743), np.float64(5.148987)]
image:  7
[-0.12695737] [